# Setting up

try installing in terminal

In [1]:
# !conda install -c conda-forge pyomo
# !conda install -c conda-forge ipopt glpk

Assume we have two banks

- A: 0.03 for (0, 10000) -> 0.015 for (10000,50000) -> 0.05 for (50000, inf)
- B: 0.005 for (0, 100000) -> 0.05 for (100000, 500000) -> 0.005 for (500000, inf)

# Idea

For example A: 0.03 for (0, 10000) -> 0.015 for (10000,50000) -> 0.05 for (50000, inf)
For Bank A, formulate the amount of asset by $(\delta_{11},\delta_{12},\delta_{13})$ as follows:
$$
f_A(\delta_{11},\delta_{12},\delta_{13}) = 0.03\delta_{11} + 0.015\delta_{12} + 0.005\delta_{13}
$$
subject to
$$
10000 w_{11} \leq \delta_{11} \leq 10000\\
40000 w_{12} \leq \delta_{12} \leq 40000w_{11}\\
0 \leq \delta_{13} \leq Xw_{12}
$$
where $X$ is principal, and $w_{11}, w_{12} \in \{0,1\}$

The objective function is maximizing $f_A + f_B$

In [2]:
import pyomo.environ as pyo
import re

In [5]:
def optimizeMyPort(principle):
      m = pyo.ConcreteModel()

      obj = 0

      #LH B-You Wealth
      m.w_LH_1           = pyo.Var(within=pyo.Binary)
      m.w_LH_2           = pyo.Var(within=pyo.Binary)
      m.w_LH_3           = pyo.Var(within=pyo.Binary)
      m.w_LH_4           = pyo.Var(within=pyo.Binary)
      m.w_LH_5           = pyo.Var(within=pyo.Binary)
      m.del_LH_1         = pyo.Var(bounds=(0,principle))
      m.del_LH_2         = pyo.Var(bounds=(0,principle))
      m.del_LH_3         = pyo.Var(bounds=(0,principle))
      m.del_LH_4         = pyo.Var(bounds=(0,principle))
      m.del_LH_5         = pyo.Var(bounds=(0,principle))
      m.del_LH_6         = pyo.Var(bounds=(0,principle))
      bound = 100000
      m.const_LH_11      = pyo.Constraint(expr = m.w_LH_1*bound <= m.del_LH_1 )
      m.const_LH_12      = pyo.Constraint(expr =                   m.del_LH_1 <= bound )
      bound = 900000-100000
      m.const_LH_21      = pyo.Constraint(expr = m.w_LH_2*bound <= m.del_LH_2 )
      m.const_LH_22      = pyo.Constraint(expr =                   m.del_LH_2 <= bound*m.w_LH_1 )
      bound = 1000000-900000
      m.const_LH_31      = pyo.Constraint(expr = m.w_LH_3*bound <= m.del_LH_3 )
      m.const_LH_32      = pyo.Constraint(expr =                   m.del_LH_3 <= bound*m.w_LH_2 )
      bound = 3000000-1000000
      m.const_LH_41      = pyo.Constraint(expr = m.w_LH_4*bound <= m.del_LH_4 )
      m.const_LH_42      = pyo.Constraint(expr =                   m.del_LH_4 <= bound*m.w_LH_3 )
      bound = 100000000-3000000
      m.const_LH_51      = pyo.Constraint(expr = m.w_LH_5*bound <= m.del_LH_5 )
      m.const_LH_52      = pyo.Constraint(expr =                   m.del_LH_5 <= bound*m.w_LH_5 )
      m.const_LH_61      = pyo.Constraint(expr =                   m.del_LH_6 <= principle*m.w_LH_5 )
      obj += 0.0025*m.del_LH_1 + 0.0175*m.del_LH_2 + 0.0555*m.del_LH_3 + 0.015*m.del_LH_4 + 0.0025*m.del_LH_5

      #KKP Start Saving
      m.w_KPP_1           = pyo.Var(within=pyo.Binary)
      m.w_KPP_2           = pyo.Var(within=pyo.Binary)
      m.w_KPP_3           = pyo.Var(within=pyo.Binary)
      m.w_KPP_4           = pyo.Var(within=pyo.Binary)
      m.w_KPP_5           = pyo.Var(within=pyo.Binary)
      m.del_KPP_1         = pyo.Var(bounds=(0,principle))
      m.del_KPP_2         = pyo.Var(bounds=(0,principle))
      m.del_KPP_3         = pyo.Var(bounds=(0,principle))
      m.del_KPP_4         = pyo.Var(bounds=(0,principle))
      m.del_KPP_5         = pyo.Var(bounds=(0,principle))
      bound = 5000
      m.const_KPP_11      = pyo.Constraint(expr = m.w_KPP_1*bound <= m.del_KPP_1 )
      m.const_KPP_12      = pyo.Constraint(expr =                m.del_KPP_1 <= bound )
      bound = 10000-5000
      m.const_KPP_21      = pyo.Constraint(expr = m.w_KPP_2*bound <= m.del_KPP_2 )
      m.const_KPP_22      = pyo.Constraint(expr =                m.del_KPP_2 <= bound*m.w_KPP_1 )
      bound = 50000-10000
      m.const_KPP_31      = pyo.Constraint(expr = m.w_KPP_3*bound <= m.del_KPP_3 )
      m.const_KPP_32      = pyo.Constraint(expr =                m.del_KPP_3 <= bound*m.w_KPP_2 )
      bound = 1500000-50000
      m.const_KPP_41      = pyo.Constraint(expr = m.w_KPP_4*bound <= m.del_KPP_4 )
      m.const_KPP_42      = pyo.Constraint(expr =                m.del_KPP_4 <= bound*m.w_KPP_3 )
      m.const_KPP_51      = pyo.Constraint(expr =                m.del_KPP_5 <= principle*m.w_KPP_4 )
      obj += 0.02*m.del_KPP_1 + 0.04*m.del_KPP_2 + 0.02*m.del_KPP_3 + 0.0155*m.del_KPP_4 + 0.005*m.del_KPP_5

      m.w_Dime_1           = pyo.Var(within=pyo.Binary)
      m.w_Dime_2           = pyo.Var(within=pyo.Binary)
      m.w_Dime_3           = pyo.Var(within=pyo.Binary)
      m.del_Dime_1         = pyo.Var(bounds=(0,principle))
      m.del_Dime_2         = pyo.Var(bounds=(0,principle))
      m.del_Dime_3         = pyo.Var(bounds=(0,principle))
      bound = 10000
      m.const_Dime_11      = pyo.Constraint(expr = m.w_Dime_1*bound <= m.del_Dime_1 )
      m.const_Dime_12      = pyo.Constraint(expr =                m.del_Dime_1 <= bound )
      bound = 1000000-10000
      m.const_Dime_21      = pyo.Constraint(expr = m.w_Dime_2*bound <= m.del_Dime_2 )
      m.const_Dime_22      = pyo.Constraint(expr =                m.del_Dime_2 <= bound*m.w_Dime_1 )
      m.const_Dime_31      = pyo.Constraint(expr =                m.del_Dime_3 <= principle*m.w_Dime_2 )
      obj += 0.03*m.del_Dime_1 + 0.015*m.del_Dime_2 + 0.005*m.del_Dime_3

      m.w_CIMBchill_1           = pyo.Var(within=pyo.Binary)
      m.w_CIMBchill_2           = pyo.Var(within=pyo.Binary)
      m.w_CIMBchill_3           = pyo.Var(within=pyo.Binary)
      m.w_CIMBchill_4           = pyo.Var(within=pyo.Binary)
      m.del_CIMBchill_1         = pyo.Var(bounds=(0,principle))
      m.del_CIMBchill_2         = pyo.Var(bounds=(0,principle))
      m.del_CIMBchill_3         = pyo.Var(bounds=(0,principle))
      m.del_CIMBchill_4         = pyo.Var(bounds=(0,principle))
      bound = 10000
      m.const_CIMBchill_11      = pyo.Constraint(expr = m.w_CIMBchill_1*bound <= m.del_CIMBchill_1 )
      m.const_CIMBchill_12      = pyo.Constraint(expr =                m.del_CIMBchill_1 <= bound )
      bound = 50000-10000
      m.const_CIMBchill_21      = pyo.Constraint(expr = m.w_CIMBchill_2*bound <= m.del_CIMBchill_2 )
      m.const_CIMBchill_22      = pyo.Constraint(expr =                m.del_CIMBchill_2 <= bound*m.w_CIMBchill_1 )
      bound = 100000-50000
      m.const_CIMBchill_31      = pyo.Constraint(expr = m.w_CIMBchill_3*bound <= m.del_CIMBchill_3 )
      m.const_CIMBchill_32      = pyo.Constraint(expr =                m.del_CIMBchill_3 <= bound*m.w_CIMBchill_2 )
      m.const_CIMBchill_41      = pyo.Constraint(expr =                m.del_CIMBchill_4 <= principle*m.w_CIMBchill_3 )
      obj += 0.005*m.del_CIMBchill_1 + 0.018*m.del_CIMBchill_2 + 0.0288*m.del_CIMBchill_3 + 0.002*m.del_CIMBchill_4

      m.const_Kept_1           = pyo.Var(within=pyo.Binary)
      m.del_Kept_1         = pyo.Var(bounds=(0,principle))
      m.const_Kept_11      = pyo.Constraint(expr = m.del_Kept_1 <= principle )
      obj += 0.0175*m.del_Kept_1

      m.w_TTB_1           = pyo.Var(within=pyo.Binary)
      m.w_TTB_2           = pyo.Var(within=pyo.Binary)
      m.w_TTB_3           = pyo.Var(within=pyo.Binary)
      m.del_TTB_1         = pyo.Var(bounds=(0,principle))
      m.del_TTB_2         = pyo.Var(bounds=(0,principle))
      m.del_TTB_3         = pyo.Var(bounds=(0,principle))
      bound = 100000
      m.const_TTB_11      = pyo.Constraint(expr = m.w_TTB_1*bound <= m.del_TTB_1 )
      m.const_TTB_12      = pyo.Constraint(expr =                m.del_TTB_1 <= bound )
      bound = 1000000-100000
      m.const_TTB_21      = pyo.Constraint(expr = m.w_TTB_2*bound <= m.del_TTB_2 )
      m.const_TTB_22      = pyo.Constraint(expr =                m.del_TTB_2 <= bound*m.w_TTB_1 )
      m.const_TTB_31      = pyo.Constraint(expr =                m.del_TTB_3 <= principle*m.w_TTB_2 )
      obj += 0.022*m.del_TTB_1 + 0.016*m.del_TTB_2 + 0.012*m.del_TTB_3

      m.w_Alpha_1           = pyo.Var(within=pyo.Binary)
      m.w_Alpha_2           = pyo.Var(within=pyo.Binary)
      m.del_Alpha_1         = pyo.Var(bounds=(0,principle))
      m.del_Alpha_2         = pyo.Var(bounds=(0,principle))
      bound = 500000
      m.const_Alpha_11      = pyo.Constraint(expr = m.w_Alpha_1*bound <= m.del_Alpha_1 )
      m.const_Alpha_12      = pyo.Constraint(expr =                m.del_Alpha_1 <= bound )
      m.const_Alpha_21      = pyo.Constraint(expr =                m.del_Alpha_2 <= principle*m.w_Alpha_1 )
      obj += 0.02*m.del_Alpha_1 + 0.007*m.del_Alpha_2

      m.w_CIMBspeed_1           = pyo.Var(within=pyo.Binary)
      m.w_CIMBspeed_2           = pyo.Var(within=pyo.Binary)
      m.w_CIMBspeed_3           = pyo.Var(within=pyo.Binary)
      m.del_CIMBspeed_1         = pyo.Var(bounds=(0,principle))
      m.del_CIMBspeed_2         = pyo.Var(bounds=(0,principle))
      m.del_CIMBspeed_3         = pyo.Var(bounds=(0,principle))
      bound = 100000
      m.const_CIMBspeed_11      = pyo.Constraint(expr = m.w_CIMBspeed_1*bound <= m.del_CIMBspeed_1 )
      m.const_CIMBspeed_12      = pyo.Constraint(expr =                m.del_CIMBspeed_1 <= bound )
      bound = 20000000-100000
      m.const_CIMBspeed_21      = pyo.Constraint(expr = m.w_CIMBspeed_2*bound <= m.del_CIMBspeed_2 )
      m.const_CIMBspeed_22      = pyo.Constraint(expr =                m.del_CIMBspeed_2 <= bound*m.w_CIMBspeed_1 )
      m.const_CIMBspeed_31      = pyo.Constraint(expr =                m.del_CIMBspeed_3 <= principle*m.w_CIMBspeed_2 )
      obj += 0.008*m.del_CIMBspeed_1 + 0.0188*m.del_CIMBspeed_2

      asset_var         = [m.del_LH_1,m.del_LH_2,m.del_LH_3,m.del_LH_4,m.del_LH_5,m.del_LH_6,
                        m.del_KPP_1,m.del_KPP_2,m.del_KPP_3,m.del_KPP_4,m.del_KPP_5,
                        m.del_Dime_1,m.del_Dime_2,m.del_Dime_3,
                        m.del_CIMBchill_1,m.del_CIMBchill_2,m.del_CIMBchill_3,m.del_CIMBchill_4,
                        m.del_Kept_1,
                        m.del_TTB_1,m.del_TTB_2,m.del_TTB_3,
                        m.del_Alpha_1,m.del_Alpha_2,
                        m.del_CIMBspeed_1,m.del_CIMBspeed_2,m.del_CIMBspeed_3]

      m.const_uni     = pyo.Constraint(expr = sum(asset_var) == principle)
      m.obj = pyo.Objective(expr=obj,
                        sense = pyo.maximize)

      pyo.SolverFactory("glpk").solve(m)


      interest_rate     = [0.0025,0.0175,0.0555,0.015,0.0025,0,
                        0.02,0.04,0.02,0.0155,0.005,
                        0.03,0.015,0.005,
                        0.005,0.018,0.0288,0.002,
                        0.0175,
                        0.022,0.016,0.012,
                        0.02,0.007,
                        0.008,0.0188,0]
      print(f'principle = {sum([pyo.value(x) for x in asset_var]):.0f}')
      asset_name = {}
      interest = {}
      for asset, rate in zip(asset_var, interest_rate):
            if pyo.value(asset) != 0:
                  asset_name[re.match(r'.+_(.+)_\d',asset.name).group(1)] = asset_name.get(re.match(r'.+_(.+)_\d',asset.name).group(1),0) + pyo.value(asset)
                  interest[re.match(r'.+_(.+)_\d',asset.name).group(1)] = interest.get(re.match(r'.+_(.+)_\d',asset.name).group(1),0) + pyo.value(asset)*rate

      for asset in asset_name:
            print(f'{asset:10s}: amount = {asset_name[asset]:8.0f},\tinterest = {interest[asset]:8.0f}')
      print()
      print("detial")
      for x in zip(asset_var,interest_rate):
            if pyo.value(x[0]) != 0:
                  print(f"{x[0].name:12s} = {pyo.value(x[0]):8.0f}, interest = {x[1]*pyo.value(x[0]):8.4f}")


      print(
      f"""
      max interest = {sum([pyo.value(x[0])*x[1] for x in zip(asset_var,interest_rate)]):.4f}
      """)

In [10]:
optimizeMyPort(100000)

principle = 100000
KPP       : amount =    10000,	interest =      300
Dime      : amount =    10000,	interest =      300
TTB       : amount =    80000,	interest =     1760

detial
del_KPP_1    =     5000, interest = 100.0000
del_KPP_2    =     5000, interest = 200.0000
del_Dime_1   =    10000, interest = 300.0000
del_TTB_1    =    80000, interest = 1760.0000

      max interest = 2360.0000
      
